In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.linear_model import LinearRegression # explicit class import from module
from sklearn.linear_model import LogisticRegression # explicit class import from module
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsClassifier #we know where this object comes from
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, recall_score
from sklearn.ensemble import RandomForestClassifier

from imblearn.over_sampling import SMOTE

warnings.filterwarnings('ignore') # Code for stopping warnings (deprecation warning, etc.)
pd.set_option('display.max_columns', None) # Code for showing all columns in the dateset, withoud '...' in between.

In [2]:
people = pd.read_csv("../raw_data/people_train.csv")

In [3]:
X = people.drop(columns = 'attrition')
y = people['attrition']

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X,y, random_state = 0)

In [9]:
sm = SMOTE(random_state=0)
X_train_smote, y_train_smote = sm.fit_sample(X_train, y_train)

In [10]:
from collections import Counter
print('before SMOTE :' , Counter(y_train))
print('after SMOTE :' , Counter(y_train_smote))

before SMOTE : Counter({0: 646, 1: 125})
after SMOTE : Counter({0: 646, 1: 646})


In [11]:
#sem smote
model = RandomForestClassifier(n_estimators=100)
sacc = cross_val_score(model, X_train, y_train, scoring='accuracy')
sf1 = cross_val_score(model, X_train, y_train, scoring='f1')
sre = cross_val_score(model, X_train, y_train, scoring='recall')
print('accuracy: ',sacc.mean())
print('f1: ',sf1.mean())
print('recall: ',sre.mean())

accuracy:  0.8521323837452869
f1:  0.19552824714115036
recall:  0.12000000000000002


In [12]:
#com smote
model = RandomForestClassifier(n_estimators=100)
sacc = cross_val_score(model, X_train_smote, y_train_smote, scoring='accuracy')
sf1 = cross_val_score(model, X_train_smote, y_train_smote, scoring='f1')
sre = cross_val_score(model, X_train_smote, y_train_smote, scoring='recall')
print('accuracy: ',sacc.mean())
print('f1: ',sf1.mean())
print('recall: ',sre.mean())


accuracy:  0.9088324204603275
f1:  0.8674849276413734
recall:  0.8434347048300538
